## Trapezoidal  Rule
__MATH 420__ <br>
_Spring 2021_ <br>

Here is a Julia implementation of the n-panel trapezoidal rule.  

In [ ]:
function trapezodial_rule(F::Function, a::Number, b::Number, n::Integer)
    h = (b-a)/n #step size--negative is OK
    s = (F(a) + F(b))/2 #weights at endpoints is 1/2
    for k=1:n-1
      s += F(a+h*k)
    end
    h*s
end

In [ ]:
 trapezodial_rule(x -> 1,0,1,1)

In [ ]:
 trapezodial_rule(x -> 1,0,1,100)

In [ ]:
 trapezodial_rule(x -> 2x, 0,1,1)

In [ ]:
 trapezodial_rule(x -> 2x, 0,1,100)

If fast was our main goal, should replace ` F(a+h*k)` by updating the knot by adding `h` after each function evaluation

In [ ]:
function trapezodial_rule2(F::Function, a::Number, b::Number, n::Integer)
    h = (b-a)/n #step size--negative is OK
    s = (F(a) + F(b))/2 #weights at endpoints is 1/2
    a = a+h #overwrite a--that's OK!
    for k=1:n-1
      s += F(a)
      a += h
    end
    @show(a)
    h*s
end

Is it really any faster? Let's try
$$
 \int_0^{10} \exp(x) \, \mathrm{d} x = \exp(10) - 1
$$

Yes, __but__

In [ ]:
 I1 = @time trapezodial_rule(exp, 0,10,10^7)

In [ ]:
I2 = @time trapezodial_rule2(exp, 0,10.0,10^7)

But wait! The values are different!  Which is more accurate? Subtracting the true value, we see that the slow and steady wins the race! The error using `trapezodial_rule` is _far less_ than the error for `trapezodial_rule2`

In [ ]:
I1 - (exp(10.0) -1)

In [ ]:
I2 - (exp(10.0) -1)

What's the story? By the time `trapezodial_rule2` evaluates the last knot (in this case 10), it has computed this value with $10^7$ additions. Each addition gives an error bounded above by the machine epsilon. After doing this many additions, the value can differ a fair amount from its true value. To see this experimentally, add a line that shows the final value of $a$ to `trapezodial_rule2` 

In [ ]:
function trapezodial_rule2(F::Function, a::Number, b::Number, n::Integer)
    h = (b-a)/n #step size--negative is OK
    s = (F(a) + F(b))/2 #weights at endpoints is 1/2
    a = a+h #overwrite a--that's OK!
    for k=1:n-1
      s += F(a)
      a += h
    end
    @show(a)
    h*s
end

In [ ]:
I2 = @time trapezodial_rule2(exp, 0,10.0,10^7)

In [ ]:
using Gadfly

In [ ]:
xx = [n for n=10:50];

In [ ]:
yy  = [trapezodial_rule(x -> x*exp(x)/exp(10),0,10,n) for n = 10:50];

In [ ]:
 plot(x=xx,y=yy,color=[colorant"blue"])

In [ ]:
I = 9 + exp(-10)

In [ ]:
yyy = [n^2 * (yy[n-9] - I) for n=10:50];

In [ ]:
plot(x=xx,y=yyy,color=[colorant"blue"])

In [ ]:
xxxx = [n for n = 10 : 25];

In [ ]:
yyyy = [ (4* yy[2*(n-9)] - yy[n-9])/3 for n=10:25]

In [ ]:
 plot(layer(x=xxxx,y=yyyy,Geom.point,color=[colorant"red"]), 
      layer(x=xx,y=yy, Geom.point,color=[colorant"blue"]))

In [ ]:
xx = [n for n=10:50];

In [ ]:
yy  = [trapezodial_rule(x -> sin(x)^2,0,pi,n) for n = 10:50]

This time, the graph doesn't look predictable--it looks like a jumpy mess!

In [ ]:
plot(x=xx,y=yy,color=[colorant"blue"])

Let's look at a graph of the difference between the trapezoidal rule value and the true value:

In [ ]:
yy  = [trapezodial_rule(x -> sin(x)^2,0,pi,n) - pi/2 for n = 10:50];

In [ ]:
plot(x=xx,y=yy,color=[colorant"blue"])

In [ ]:
using ForwardDiff